  
  
  
  


# Shiny App: Song Suggester, part II: Building the app in R.

***
## About the Project

This is the follow-up project for the [API calls code](https://github.com/zefrios/Python/blob/8d958c935980235939b70a5005aba2de9a547961/Spotify_APIcalls/SpotifyTokenRequest.ipynb) we did on Python.

Now it is easier to work in R. There, the program will focus mostly on creating a UI and leveraging the dataframes from our previous Python code. The app allows users to enter an artist's name and select a geographical market, then uses Python to fetch related artists and their top tracks. Then, it processes the returned data to display the most popular tracks and a list of related artists in an interactive web interface.


First, we declare the necessary libraries:

In [ ]:
library(shiny)
library(reticulate)
library(dplyr)
library(DT)

Secondly, the code from Python is called, the elements the we will find in the UI are created and set the elements' output commands for the incoming tables. For simplicity, 8 markets were added nto the list to compare if there were changes in the recommended songs between regions.

In [ ]:
source_python("api_calls_Deploy.py")

ui <- fluidPage(
  titlePanel("Spotify Related Artists and Tracks"),
  textInput("artistName", "Enter Artist Name:"),
  selectInput("marketSelect", "Select Market:",
              choices = c("AU", "CA", "US", "GB", "DE", "FR", "JP", "MX"),  # Add more markets as needed
              selected = ""),
  actionButton("searchButton", "Search"),
  DTOutput("topTracksTable"),
  tableOutput("relatedArtistsTable")
)

Finally, on the server function we make sure we are receiving the two dataframes output by out Python code and then we render the output tables.

In [ ]:
server <- function(input, output, session) {
  
  observeEvent(input$searchButton, {
    req(input$artistName)
    market <- input$marketSelect
    
    print(paste("Selected market:", market))
    
    artist_data <- py$main(input$artistName, market)
    
    if (is.list(artist_data) && length(artist_data) >= 1) {
      # Ensure the first element is a data frame
      if (is.data.frame(artist_data[[1]])) {
        top_tracks_df <- artist_data[[1]]
        
        print(str(top_tracks_df))
        top_tracks_df$track_url <- sprintf('<a href="%s" target="_blank">Link</a>', top_tracks_df$track_url)
        
        # Apply dplyr filtering
        processed_tracks_df <- top_tracks_df %>%
          arrange(desc(popularity)) %>%
          group_by(artist_name) %>%
          slice_head(n = 1) %>%
          ungroup()
        
        
        print(str(processed_tracks_df))
        print(head(processed_tracks_df))
        
        # Render the top tracks table
        output$topTracksTable <- renderDT({
          processed_tracks_df
        }, escape = FALSE)
      }
      
      # Check and render the related artists table
      if (length(artist_data) >= 2 && is.data.frame(artist_data[[2]])) {
        related_artists_df <- artist_data[[2]]
        output$relatedArtistsTable <- renderTable({
          related_artists_df
        })
      }
    } else {
      print("Error: Unexpected or no data returned from Python script.")
    }
  })
}

We run the app with the following command:

In [ ]:
shinyApp(ui, server)

The final result. Let's try to search for a band. Since I'm a fan, Black Sabbath sounds like a good option.

![Im](RShiny_SpotifyApp_2.png)

Note that we can search for the same artist in different markets. Let's try Mexico this time. There is a slight difference on the most popular songs per artist between different markets. For example, in Australia the most popular song by Black Label Society is *Funeral Bell*, with a popularity index of 57. On the other hand, in Mexico, their most popular tune is *Stillborn* with a popularity index of 68.

![Image2](RShiny_SpotifyApp_3.png)

It is also possible to listen to the recommended song via the web's interface. 

![Image3](RShiny_SpotifyApp_4.png)

## Next steps

Since it seems like our dataset could give us some interesting insights on how the dataframes from Spotify are organized and which parameters could give us a more detailed view on these popular songs, an EDA with Spotify's data will be conducted. The project can be found here: [Spotify API EDA: Black Sabbath](https://github.com/zefrios/SQL/blob/b0c18f1bb41104a08b457e7e347c9d08785f355c/Black%20Sabbath%20EDA/README.md)

***